# Imports

In [1]:
import pandas as pd
import numpy as np

---

# Carga de datos

In [2]:
auctions = pd.read_pickle("../../../../data/tp2/auctions_tp2_formateado.pkl")

---

# Armado de ventanas

In [3]:
for ventana_nro in range(1,6):
    auctions["ventana_{}".format(ventana_nro)] = (((17+ventana_nro) <= auctions["date"].dt.day) & ((auctions["date"].dt.day) <= (19+ventana_nro)))

---

# Creación de features

## Creación de sets de entrenamiento

In [4]:
Xs = {}
for ventana_nro in range(1,6):
    Xs[ventana_nro] = auctions.loc[auctions["ventana_{}".format(ventana_nro)]]["device_id"].drop_duplicates().to_frame().set_index("device_id")

---

## Función generadora de features

In [134]:
def generar_feature_en_ventanas(dataframe, generador_feature, destinos):
    for ventana_nro in range(1, 6):
        feature = generador_feature(dataframe.loc[dataframe["ventana_{}".format(ventana_nro)]], ventana_nro)
        destinos[ventana_nro] = destinos[ventana_nro].merge(feature, left_index=True, right_index=True)

---

## Cantidad de apariciones en subastas dentro de la ventana

In [6]:
def cantidad_apariciones_en_subastas(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"date" : "count"}).rename(columns={"date": "cantidad_apariciones"})

In [7]:
generar_feature_en_ventanas(auctions, cantidad_apariciones_en_subastas, Xs)

---

## Cantidad de apariciones discriminando períodos

In [8]:
# Creo columnas para luego separar
auctions['hora'] = auctions['date'].dt.hour
auctions['dia'] = auctions['date'].dt.day
auctions['dia_de_la_semana'] = auctions['date'].dt.dayofweek

In [9]:
def cantidad_apariciones_por_hora(dataframe, nro_ventana):
    apariciones_por_hora = dataframe.groupby(["device_id", "hora"]).agg(({"date" : "count"}))
    apariciones_por_hora = apariciones_por_hora.unstack().fillna(0).astype(np.int64)
    apariciones_por_hora.columns = ["apariciones_en_hora_{}".format(hora) for hora in range(0,24)]
    return apariciones_por_hora

In [10]:
generar_feature_en_ventanas(auctions, cantidad_apariciones_por_hora, Xs)

In [11]:
def cantidad_apariciones_por_dia(dataframe, nro_ventana):
    dias = dataframe["dia"].drop_duplicates
    apariciones_por_dia = dataframe.groupby(["device_id", "dia"]).agg(({"date" : "count"}))
    apariciones_por_dia = apariciones_por_dia.unstack().fillna(0).astype(np.int64)
    apariciones_por_dia.columns = ["apariciones_en_dia_{}".format(dia) for dia in range(1,4)]
    return apariciones_por_dia

In [12]:
generar_feature_en_ventanas(auctions, cantidad_apariciones_por_dia, Xs)

---

## Sistema operativo (Android / iOS)

In [13]:
def cantidad_so_registrados(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"ref_type_id" : "nunique"}).rename(columns={"ref_type_id": "cantidad_so_registrados"})

In [14]:
generar_feature_en_ventanas(auctions, cantidad_so_registrados, Xs)

In [ ]:
# Tarda MUCHO
def sistema_operativo_principal(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"ref_type_id" : (lambda x: x.value_counts().index[0])}).rename(columns={"ref_type_id": "so_mas_usado"})

In [ ]:
generar_feature_en_ventanas(auctions, sistema_operativo_principal, Xs)

In [16]:
def media_so(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"ref_type_id" : "mean"}).rename(columns={"ref_type_id": "so_media"})

In [17]:
generar_feature_en_ventanas(auctions, media_so, Xs)

---

## Tiempo medio entre apariciones

In [46]:
def tiempo_medio_entre_apariciones(dataframe, nro_ventana):
    gb = dataframe.groupby("device_id")
    ps = {}
    for t, g in gb:
        ps[t] = g["date"].sub(g["date"].shift()).mean()
    return pd.Series(ps).dt.total_seconds().fillna(0).to_frame().rename(columns={0: "tiempo_medio_entre_apariciones"})

In [47]:
generar_feature_en_ventanas(auctions, tiempo_medio_entre_apariciones, Xs)

---

## Tiempo máximo entre apariciones

In [55]:
def tiempo_maximo_entre_apariciones(dataframe, nro_ventana):
    gb = dataframe.groupby("device_id")
    ps = {}
    for t, g in gb:
        ps[t] = g["date"].sub(g["date"].shift()).max()
    return pd.Series(ps).dt.total_seconds().fillna(0).to_frame().rename(columns={0: "tiempo_maximo_entre_apariciones"})

In [56]:
generar_feature_en_ventanas(auctions, tiempo_maximo_entre_apariciones, Xs)

---

## Desvío entre apariciones

In [123]:
# Tarda MUCHO
def desvio_entre_apariciones(dataframe, nro_ventana):
    gb = dataframe.groupby("device_id")
    ps = {}
    for t, g in gb:
        ps[t] = g["date"].sub(g["date"].shift()).std()
    return pd.Series(ps).dt.total_seconds().fillna(0).to_frame().rename(columns={0: "desvio_entre_apariciones"})

In [133]:
generar_feature_en_ventanas(auctions, desvio_entre_apariciones, Xs)

---

## Cantidad de apariciones por fuente

In [92]:
def cantidad_apariciones_segun_source(dataframe, nro_ventana):
    feature = dataframe.groupby(["device_id", "source_id"])["date"].agg("count").to_frame().unstack()
    feature.columns = ["apariciones_en_source_{}".format(x) for x in range(10)]
    feature = feature.fillna(0).astype(np.int64)
    return feature

In [93]:
generar_feature_en_ventanas(auctions, cantidad_apariciones_segun_source, Xs)

---

## Tiempo desde la última aparición hasta el fin de la ventana

In [118]:
def tiempo_desde_ult_aparicion_hasta_fin_ventana(dataframe, nro_ventana):
    return (np.datetime64("2019-04-2{}".format(nro_ventana)) - dataframe.groupby("device_id").agg({"date" : "max"})["date"]).dt.total_seconds().to_frame().rename(columns={"date": "tiempo_ultima_aparicion_hasta_fin_ventana"})

In [119]:
generar_feature_en_ventanas(auctions, tiempo_desde_ult_aparicion_hasta_fin_ventana, Xs)

---

# Guardado de los sets de entrenamiento

In [136]:
for ventana_nro in range(1, 6):
    Xs[ventana_nro].to_pickle("../../features/auctions_train_ventana_{}.pkl".format(ventana_nro))
    Xs[ventana_nro].to_csv("../../features/auctions_train_ventana_{}.csv".format(ventana_nro))

---